In [2]:
import attr
import astropy.units as u
import numpy as np

In [32]:
@attr.s
class galaxy:
    x_ = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    y_ = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    z_ = attr.ib(default = 0.*(u.km/u.s), validator=attr.validators.instance_of(u.Quantity))
    r_ = attr.ib(default=None)
    
    def __change_units_to_array__(f):
        def new_func(self,*args,**kwargs):    
            self.x = self.x_.to_value(u.kpc)
            self.y = self.y_.to_value(u.M_sun)
            self.z = self.z_.to_value(u.km/u.s)
            return f(self,*args,**kwargs)
        return new_func
    
    @__change_units_to_array__
    def func(self, s=2):
        w = 2* self.x * self.y
        return w
    
    @__change_units_to_array__
    def func2(self):
        q = 2*self.x
        return q

In [33]:
yy = galaxy(x_=5*u.kpc, y_=3*u.M_sun)
yy

galaxy(x_=<Quantity 5. kpc>, y_=<Quantity 3. solMass>, z_=<Quantity 0. km / s>, r_=None)

In [34]:
yy.x_

<Quantity 5. kpc>

In [35]:
yy.func()

30.0

In [36]:
yy.func2()

10.0

In [38]:
type(yy.x)

numpy.float64

In [15]:
5*u.Mpc + 300*u.kpc

<Quantity 5.3 Mpc>

In [17]:
type(np.asarray(y.func()))

numpy.ndarray

In [44]:
type(y.x)

astropy.units.quantity.Quantity

In [43]:
y.x

<Quantity 5000. kpc>

In [25]:
u.ma

AttributeError: module 'astropy.units' has no attribute 'mass'

In [40]:
@u.quantity_input(a=['length', 'speed'])
def myfunction(a):
    return a.to_value(u.kpc)

In [41]:
myfunction(5*u.Mpc)

5000.0

In [55]:
@attr.s
class galaxy:
    x = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    y = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    
    @u.quantity_input(a=['length'])
    def myfunction(self):
        return self.a.to_value(u.kpc)
    
    @u.quantity_input(a=['mass'])
    def myfunction2(self):
        return self.a.to_value(u.Msun)



In [52]:
yy=galaxy(y=3e20*u.kg,x=5*u.Mpc)

In [38]:
yy.x.unit.physical_type


'length'

In [5]:
y = 3*(u.m/u.s)**2

In [6]:
y.unit.physical_type

'unknown'

In [50]:
G = 4.299e-6 * u.kpc * (u.km / u.s) ** 2 / u.M_sun

In [55]:
G.to_value()

4.299e-06

In [16]:
import attr
import astropy.units as u
import numpy as np

@attr.s()
class UnitConverter:
    
    default_unit = attr.ib()
    
    def __call__(self, v):
        if isinstance(v, u.Quantity) and v.unit != u.dimensionless_unscaled:
            return v
        return v * self.default_unit
    
    
def unit_attribute(valid_units, default_unit, **kwargs):
    if "converter" in kwargs:
        raise AttributeError("converter")
    converter = UnitConverter(default_unit=default_unit)
        
    def _unit_validator(instance, attribute, value):
        if value.unit.physical_type not in valid_units:
            raise ValueError(f"Attribute '{attribute.name}' must have units of {valid_units}. Found {value.unit}")
    
    validators = kwargs.pop("validator", [])
    validators.append(_unit_validator)
    
    return attr.ib(validator=validators, converter=converter, **kwargs)
    
    
@attr.s()
class Galaxy:
    p = unit_attribute(valid_units=['length'], default_unit=(u.kpc))
    
    

In [100]:
import attr
import astropy.units as u
import numpy as np

@attr.s()
class UnitConverter:
    
    default_unit = attr.ib()
    
    def __call__(self, v):
        if isinstance(v, u.Quantity) and v.unit != u.dimensionless_unscaled:
            return v
        return v * self.default_unit
    
    
def unit_attribute(valid_units, default_unit, **kwargs):
    if "converter" in kwargs:
        raise AttributeError("converter")
    converter = UnitConverter(default_unit=default_unit)
        
    def _unit_validator(instance, attribute, value):
        if value.unit.physical_type not in valid_units:
            raise ValueError(f"Attribute '{attribute.name}'"+
                             "must have units of {valid_units}. Found {value.unit}")
    
    validators = kwargs.pop("validator", [])
    validators.append(_unit_validator)
    
    return attr.ib(validator=validators, converter=converter, **kwargs)
    
    
@attr.s()
class Galaxy:
    p = unit_attribute(valid_units=['length'], default_unit=(u.kpc))
    y = unit_attribute(default = 0 ,valid_units=['length'], default_unit=(u.kpc))
    
    def energy(self):
        p = self.p.to_value(u.kpc)
        q = p**2
        setattr(self, 'q', q*(self.p.unit)**2)
        return q,
        q,
        q
    

In [19]:
import uttr
import attr
import astropy.units as u
import numpy as np

In [34]:
@attr.s()
class Galaxy:
    p = uttr.ib(unit=u.kpc)
    y = uttr.ib(default = 0 ,unit=u.kpc)
    
    arr_ = uttr.array_accessor()
    
    def energy(self):
        p = self.p.to_value(u.kpc)
        q = p**2
        setattr(self, 'q', q*(self.p.unit)**2)
        return q

In [35]:
yy = Galaxy(p=np.array([5,2])*u.km)
yy

Galaxy(p=<Quantity [5., 2.] km>, y=<Quantity 0. kpc>)

In [41]:
yy.p.to_value(u.kpc)

array([1.62038964e-16, 6.48155858e-17])

In [39]:
yy.arr_.p

array([1.62038964e-16, 6.48155858e-17])

In [65]:
q = u.Quantity(500,unit=u.Msun*(u.km/u.s)**2)

In [66]:
q.unit.physical_type

'energy'

In [118]:
# Using @property decorator
@attr.s
class galaxy:
    x = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    y = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    z = attr.ib(default = 0.*(u.km/u.s), validator=attr.validators.instance_of(u.Quantity))
    r = attr.ib(default=None)
    
    @property
    def __change_unit(self):
        sefl.x_ = self.x.to_value(u.kpc)
        self.y_ = self.y.to_value(u.M_sun)
        self.z_ = self.z.to_value(u.km/u.s)
        return self
    
    @property
    def func(self,*args,**kwargs):
        w = 2* self.x_ * self.y_
        return w
    
    @property
    def func2(self,*args,**kwargs):
        q = 2*self.x_
        return q
    

In [119]:
g = galaxy(x=5*u.Mpc, y=10*u.M_sun)

In [120]:
g

galaxy(x=<Quantity 5. Mpc>, y=<Quantity 10. solMass>, z=<Quantity 0. km / s>, r=None)

In [121]:
g.func()

AttributeError: 'galaxy' object has no attribute 'x_'

In [ ]:
def __change_units_to_array__(f):
        def new_func(self,*args,**kwargs):    
            self.x = self.x_.to_value(u.kpc)
            self.y = self.y_.to_value(u.M_sun)
            self.z = self.z_.to_value(u.km/u.s)
            return f(self,*args,**kwargs)
        return new_func

In [ ]:
class galaxy:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    @property
    def multiplicacion (self):
        return (self.x * self.y)

In [ ]:
@attr.s
class galaxy:
    x = attr.ib()
    y = attr.ib()
    
    @property
    def multiplicacion (self):
        return (self.x * self.y)

In [21]:
import astropy.units as u
@u.quantity_input(myangle=u.arcsec)
def myfunction(myangle):
    return myangle**2
    

In [24]:
myfunction(0.5*u.kg)

UnitsError: Argument 'myangle' to function 'myfunction' must be in units convertible to 'arcsec'.